In [1]:
import mne
from mne.datasets import sample
from mne import io
from mne.preprocessing import create_ecg_epochs, create_eog_epochs
from mne.preprocessing import ICA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
from os import path
import fnmatch


In [2]:
#test=mne.io.read_raw_bdf('/home/dcellier/RDSS/AlphaStudy_Data/eegData/eeg_eyetracking/extra_or_defunct/sub_212_eyetracking_testing.bdf',montage=mne.channels.read_montage('biosemi64'),preload=True)

In [3]:
#test.plot()

In [4]:
ROOT_eeg='/home/dcellier/RDSS/AlphaStudy_Data/eegData/eeg_raw/'
ROOT_behav='/home/dcellier/RDSS/AlphaStudy_Data/eegData/eeg_behavior_data/'
ROOT_eyetrack='/data/backed_up/shared/AlphaStudy_data/placeholders/eyetracking_compiled_trials/'

In [7]:
#testing_sub=mne.io.read_raw_bdf(ROOT_eeg+'sub_245_alpha_placeholders_02_04_20.bdf',montage=mne.channels.read_montage('biosemi64'),preload=True)
testing_sub=mne.io.read_raw_fif(ROOT_eeg+'sub_245_alpha_placeholders_02_04_20.bdf',preload=True)

Opening raw data file /home/dcellier/RDSS/AlphaStudy_Data/eegData/eeg_raw/sub_245_alpha_placeholders_02_04_20.bdf...


<ipython-input-7-043e43e28a38>:2: RuntimeWarning: This filename (/home/dcellier/RDSS/AlphaStudy_Data/eegData/eeg_raw/sub_245_alpha_placeholders_02_04_20.bdf) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
  testing_sub=mne.io.read_raw_fif(ROOT_eeg+'sub_245_alpha_placeholders_02_04_20.bdf',preload=True)


    Range : 0 ... 2152959 =      0.000 ...  4204.998 secs
Ready.
Reading 0 ... 2152959  =      0.000 ...  4204.998 secs...


In [8]:
testing_sub.info

<Info | 19 non-empty fields
    bads : list | 0 items
    ch_names : list | Fp1, AF7, AF3, F1, F3, F5, F7, FT7, FC5, ...
    chs : list | 73 items (EEG: 72, STIM: 1)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : Transform | 3 items
    dig : list | 67 items (3 Cardinal, 64 EEG)
    events : list | 0 items
    file_id : dict | 4 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 104.0 Hz
    meas_date : tuple | 2020-02-04 17:19:04 GMT
    meas_id : dict | 4 items
    nchan : int | 73
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 512.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    experimenter : NoneType
    gantry_angle : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    proj_id : NoneType
    proj_name : NoneType
   

In [9]:
behav_files=pd.DataFrame()
all_files=[]
for f in os.listdir(ROOT_behav):
    if '255_' in f:
        print(f)
        behav_file=pd.read_csv(ROOT_behav+f,engine='python')
        all_files.append(behav_file)
        behav_files=behav_files.append(behav_file,ignore_index=True)
#behav_files.dropna(axis=0,inplace=True,how='any')
behav_files=behav_files.sort_values(['block','trialNum']).reset_index() 
#print(behav_files)
#behav_files.to_csv('/home/dcellier/RDSS/AlphaStudy_Data/eegData/eeg_behavior_data/sub254_TRUNCATED_TO_MATCH_probeOnly.csv')

255_alpha_placeholders_matchingBrad_01_1_2020_Feb_24_1406.csv
255_alpha_placeholders_matchingBrad_01_2_2020_Feb_24_1418.csv
255_alpha_placeholders_matchingBrad_01_3_2020_Feb_24_1425.csv
255_alpha_placeholders_matchingBrad_01_6_2020_Feb_24_1445.csv
255_alpha_placeholders_matchingBrad_01_7_2020_Feb_24_1451.csv
255_alpha_placeholders_matchingBrad_01_8_2020_Feb_24_1457.csv
255_alpha_placeholders_matchingBrad_01_9_2020_Feb_24_1503.csv
255_alpha_placeholders_matchingBrad_01_10_2020_Feb_24_1509.csv


In [31]:
behav_files.columns

Index(['tar_color', 'dis_color', 'SOA', 'block', 'trialNum', 'trial_type',
       'corrResp', 'subResp', 'trialCorr?', 'RT', 'tar,dis stim_loc', 'ITI',
       'Tar,Dis,Other', 'trial_trigs', 'triggers'],
      dtype='object')

In [32]:
behav_files=behav_files.sort_values(['block','trialNum'])
#behav_files

In [33]:
np.unique(list(behav_files['block']))

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [34]:
#behav_files.to_csv('/home/dcellier/RDSS/AlphaStudy_Data/eegData/eeg_behavior_data/compiledAllBlocks_s233.csv')

In [35]:
eyetrack_compiled_path='/data/backed_up/shared/AlphaStudy_data/placeholders/eyetracking_compiled_trials/'
sub_name='227'
pattern=sub_name+'_alpha_placeholders_matchingBrad_*_*_*_*_*_*.csv'
pattern2=sub_name+'_alpha_placeholders_matchingBrad_*_*_*_*_*.csv'
pattern3=sub_name+'_alpha_placeholders_matchingBrad_*_*_*_*_*_TRUNCATED_TO_MATCH_BDF.csv'
pattern4=sub_name+'_alpha_placeholders_matchingBrad_*_*_*_*_*_*_TRUNCATED_TO_MATCH_BDF.csv'
behav_files=pd.DataFrame()
for f in os.listdir(ROOT_behav):
    if fnmatch.fnmatch(f,pattern) or fnmatch.fnmatch(f,pattern2) or fnmatch.fnmatch(f,pattern3) or fnmatch.fnmatch(f,pattern4):
        print(f)
        behav_file=pd.read_csv(ROOT_behav+f,engine='python')
        behav_files=behav_files.append(behav_file,ignore_index=True)
behav_files_sorted=behav_files.sort_values(['block','trialNum']).reset_index()
#eyes=pd.read_csv(eyetrack_compiled_path+'sub_'+sub_name+'_trialsCompiled.csv')
behav_files=pd.concat([behav_files_sorted,eyes],ignore_index=False,axis=1)

227_alpha_placeholders_matchingBrad_01_1_2020_Mar_05_1009.csv
227_alpha_placeholders_matchingBrad_01_2_2020_Mar_05_1105.csv
227_alpha_placeholders_matchingBrad_01_3_2020_Mar_05_1112.csv
227_alpha_placeholders_matchingBrad_01_4_2020_Mar_05_1120.csv


In [9]:
events=mne.find_events(testing_sub, initial_event=True,min_duration=(2/512))# 133 = start block flag, 135 = stop block

2787 events found
Event IDs: [  101   103   105   107   109   111   113   115   117   119   121   123
   125   127   129   131   133   135 65536 65737]


In [10]:
## Generating Epochs obj so I can make sure the metadata matches up
cue_event_id={'disShortCue_trig':101,'disLongCue_trig':103,'tarShortCue_trig':105,
             'tarLongCue_trig':107,'neutShortCue_trig':109, 'neutLongCue_trig':111}
testingSub_cueEps=mne.Epochs(testing_sub, events=events, baseline=None, event_id=cue_event_id,
                         tmin=-1.3,tmax=1,metadata=behav_files)

NameError: name 'behav_files' is not defined

In [11]:
probe_event_id={'disShortProbe_trig':113,'disLongProbe_trig':115,'tarShortProbe_trig':117,
                    'tarLongProbe_trig':119,'neutShortProbe_trig':121,'neutLongProbe_trig':123}
probe_tmin, probe_tmax = 0,5.4
#testingSub_probeEps=mne.Epochs(testing_sub, events=events, baseline=(2,5.4), event_id=probe_event_id, 
 #                                 tmin=probe_tmin, tmax=probe_tmax,metadata=behav_files)

In [41]:
len(behav_files)

600

In [42]:
np.sum(sub224_cueEps['disLongCue_trig'].metadata['SOA']!='long')

NameError: name 'sub224_cueEps' is not defined

In [43]:
len(events)

3020

In [13]:
## SEPERATING TRIGS BY BLOCKS
    # not all start_block trigs came through 
    # but all of the end block triggers did,
    # so we're basing off of those

blocks={}
block_counter=0
trig_counter=0
while block_counter<10: # we usually have 10 blocks of EEG
    ev=events[trig_counter]
    #print(trig_counter)

    if ev[2]==135: # then we've found the end of the block, move on to the next
        blocks[str(block_counter)].append(ev)
        block_counter += 1
    else:
        if str(block_counter) not in blocks.keys():
            blocks[str(block_counter)]=[]
        blocks[str(block_counter)].append(ev)
        
    trig_counter += 1
        

In [14]:
for k in blocks.keys():
    probe_count=len([ev[2] for ev in blocks[k] if ev[2] in [113,115,117,119,121,123]])
    if probe_count < 60:
        print('Block {0} has {1} probe events'.format(k,str(probe_count)))

Block 2 has 13 probe events


In [12]:
ITI_evs=[ev[2] for ev in events if ev[2]==127]
mask_evs=[ev[2] for ev in events if ev[2]==125]
probe_evs=[ev[2] for ev in events if ev[2] in [113,115,117,119,121,123]]
cue_evs=[ev[2] for ev in events if ev[2] in [101,103,105,107,109,111]]
resp_evs=[ev[2] for ev in events if ev[2] in [129,131]]
Startblock_evs=[ev[2] for ev in events if ev[2] ==133]
Stopblock_evs=[ev[2] for ev in events if ev[2] ==135]
print("Len ITI trigs: "+ str(len(ITI_evs)))
print("Len Mask trigs: "+ str(len(mask_evs)))
print("Len Probe trigs: "+str(len(probe_evs)))
print("Len cue events: "+str(len(cue_evs)))
print("Len responses: "+str(len(resp_evs)))
print("Len start block trigs: "+str(len(Startblock_evs)))
print("Len stop block trigs: "+str(len(Stopblock_evs)))

Len ITI trigs: 553
Len Mask trigs: 553
Len Probe trigs: 553
Len cue events: 553
Len responses: 553
Len start block trigs: 9
Len stop block trigs: 10


In [15]:
probe_block=[ev[2] for ev in blocks['2'] if ev[2] in [113,115,117,119,121,123]]
#probe_block0
enumerated_trigs=[]
for i,p in enumerate(probe_block):
    enumerated_trigs.append([i,p])

In [16]:
enumerated_trigs

[[0, 115],
 [1, 123],
 [2, 117],
 [3, 123],
 [4, 123],
 [5, 121],
 [6, 115],
 [7, 117],
 [8, 119],
 [9, 123],
 [10, 113],
 [11, 115],
 [12, 113]]

In [81]:
cue_block=[ev[2] for ev in blocks['6'] if ev[2] in [101,103,105,107,109,111]]
#probe_block0
enumerated_trigs=[]
for i,p in enumerate(cue_block):
    enumerated_trigs.append([i,p])

In [82]:
enumerated_trigs

[[0, 111],
 [1, 109],
 [2, 111],
 [3, 109],
 [4, 109],
 [5, 101],
 [6, 107],
 [7, 109],
 [8, 107],
 [9, 103],
 [10, 105],
 [11, 105],
 [12, 103],
 [13, 105],
 [14, 105],
 [15, 109],
 [16, 101],
 [17, 101],
 [18, 107],
 [19, 109],
 [20, 111],
 [21, 111],
 [22, 101],
 [23, 103],
 [24, 101],
 [25, 103],
 [26, 111],
 [27, 101],
 [28, 111],
 [29, 101],
 [30, 107],
 [31, 107],
 [32, 111],
 [33, 101],
 [34, 101],
 [35, 103],
 [36, 103],
 [37, 107],
 [38, 109],
 [39, 103],
 [40, 105],
 [41, 103],
 [42, 107],
 [43, 105],
 [44, 103],
 [45, 105],
 [46, 107],
 [47, 101],
 [48, 107],
 [49, 105],
 [50, 107],
 [51, 109],
 [52, 109],
 [53, 111],
 [54, 103],
 [55, 105],
 [56, 105],
 [57, 111],
 [58, 111],
 [59, 109]]

In [56]:
behav_trigs_probe_block0=list(behav_files[behav_files['block']==1]['trial_trigs'])
behav_trigs_probe_block0[0][5:10]
enumerated_behav=[]
for i,p in enumerate(behav_trigs_probe_block0):
    #print(i,int(p[5:9]))
    enumerated_behav.append([i,int(p[5:9])])


In [ ]:
for t,b in zip(enumerated_trigs,enumerated_behav):
    print(t,b)

In [ ]:
#cue_evs[:60]
#cue_evs[60:120]
#cue_evs[120:180]
#cue_evs[180:240]

In [ ]:
#all_trigs[:10]

In [ ]:
mask_trigs=[ev[2] for ev in events if ev[2]==125]
len(mask_trigs)

In [ ]:
all_trigs=[ev[2] for ev in events if ev[2] in [101,103,105,107,109,111,113,115,117,119,121,123]]

In [ ]:
len(all_trigs)/2

In [ ]:
#all_trigs[-10:]

In [ ]:
all_trigs[:10]

In [ ]:
#events[:10]
all_trigs[-10:]

In [ ]:
behav_files

In [ ]:
behav_files=pd.DataFrame()
sub_name='228'
pattern=sub_name+'_alpha_placeholders_matchingBrad_*_*_*_*_*_*.csv'
pattern2=sub_name+'_alpha_placeholders_matchingBrad_*_*_*_*_*.csv'
pattern3=sub_name+'_alpha_placeholders_matchingBrad_*_*_*_*_*_TRUNCATED_TO_MATCH_BDF.csv'
pattern4=sub_name+'_alpha_placeholders_matchingBrad_*_*_*_*_*_*_TRUNCATED_TO_MATCH_BDF.csv'
behav_files=pd.DataFrame()
for f in os.listdir(ROOT_behav):
    if fnmatch.fnmatch(f,pattern) or fnmatch.fnmatch(f,pattern2) or fnmatch.fnmatch(f,pattern3) or fnmatch.fnmatch(f,pattern4):
        print(f)
        behav_file=pd.read_csv(ROOT_behav+f,engine='python')
        behav_files=behav_files.append(behav_file,ignore_index=True)


In [ ]:
behav_files=behav_files.sort_values(['block','trialNum']).reset_index()

In [ ]:
behav_files.head()

In [ ]:
#eyes=pd.read_csv(ROOT_eyetrack+'sub_197_trialsCompiled_TRUNCATED_TO_MATCH.csv')
#adjusted_behav=pd.read_csv(ROOT_behav+'197_alpha_placeholders_matchingBrad_01_2019_Nov_12_1319_TRUNCATED_TO_MATCH_BDF.csv')

In [ ]:
eyes=pd.read_csv(ROOT_eyetrack+'sub_228_trialsCompiled_TRUNCATED_TO_MATCH.csv')

In [ ]:
len(eyes)

In [ ]:

eyes.head()

In [ ]:
appended=pd.concat([behav_files,eyes],ignore_index=False,axis=1)
#appended.to_csv('/data/backed_up/shared/AlphaStudy_data/placeholders/eyetracking_compiled_trials/sub_228_trialsCompiled_FOR_PROBE.csv')

In [ ]:
#behav_files.append(eyes,ignore_index=True,axis=1)
appended

In [ ]:
pd.read_csv('/data/backed_up/shared/AlphaStudy_data/placeholders/eyetracking_compiled_trials/sub_212_trialsCompiled_FOR_CUES.csv')